# Tic-Tac-Toe 
**Author:** Cameron Wonchoba

This program uses data that had been previously collected to train several models. These models are then used to predict a "smart" move for a tic-tac-toe player. This is tested against me. The model accuracy, and example games against each model can be viewed below.

In [1]:
import operator
import copy
import random

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

## tictactoe class 
This class defines a board for tictactoe. The following methods are included:

* get_row()
* get_col()
* get_board()
* set_row_col()
* valid_move()
* set_move(move)
* print_board()
* clear_board()
* winner()
* get_available_moves()
* board_as_dict()
* draw()

In [2]:
class tictactoe:

    def __init__(self):
        # Define the board
        self.row = 0
        self.col = 0
        self.board = [['-','-','-'],['-','-','-'],['-','-','-']]

    def get_row(self):
        # Return the row.
        return self.row

    def get_col(self):
        # Return the column.
        return self.col

    def get_board(self):
        # Return the state of the board.
        return self.board

    def set_row_col(self,row,col):
        # Set the row and column.
        self.row = row
        self.col = col

    def valid_move(self):
        # Check if a move is valid.
        if self.row >= 0 and self.row <= 2 and self.col >= 0 and self.col <= 2:
            return self.board[self.row][self.col] == '-'

    def set_move(self,move):
        # Set the current row and column to the move.
        self.board[self.row][self.col] = move

    def print_board(self):
        # Print the state of the board.
        for row in self.board:
            print(" ".join(row))

    def clear_board(self):
        # Clear the board.
        for i, row in enumerate(self.board):
            self.board[i] = ['-']*3

    def winner(self):
        # Check if there is a winner.
        i = 0
        while i <= 2:
            if self.board[0][i] == 'x' and self.board[1][i] == 'x'and self.board[2][i] == 'x':
                return True
            elif self.board[0][i] == 'o' and self.board[1][i] == 'o' and self.board[2][i] == 'o':
                return True
            elif self.board[i][0] == 'x' and self.board[i][1] == 'x'and self.board[i][2] == 'x':
                return True
            elif self.board[i][0] == 'o' and self.board[i][1] == 'o' and self.board[i][2] == 'o':
                return True
            i += 1
        if self.board[0][0] == 'x' and self.board[1][1] == 'x' and self.board[2][2] == 'x':
                return True
        elif self.board[0][0] == 'o' and self.board[1][1] == 'o' and self.board[2][2] == 'o':
                return True
        elif self.board[0][2] == 'x' and self.board[1][1] == 'x' and self.board[2][0] == 'x':
                return True
        elif self.board[0][2] == 'o' and self.board[1][1] == 'o' and self.board[2][0] == 'o':
                return True
        return False

    def get_available_moves(self):
        # Return a list of all available moves in the form a list.
        moves = []
        for i, row in enumerate(self.board):
            for j, col in enumerate(self.board):
                if self.board[i][j] == '-':
                    moves += [i*3+j]
        return moves

    def board_as_dict(self):
        # Return the state of the board in the form of a dictionary.
        dict_board = {}
        for i, row in enumerate(self.board):
            for j, col in enumerate(row):
                if self.board[i][j] == '-':
                    dict_board[i*3+j] = 0
                elif self.board[i][j] == 'x':
                    dict_board[i*3+j] = 1
                else:
                    dict_board[i*3+j] = 2
        return dict_board

    def draw(self):
        # Check if there is a tie.
        for row in self.board:
            if '-' in row:
                return False
        return True

## model_creation class 
This class records the history of different moves and builds a model (Decision Tree) from that data collected.
The following methods are included:

* update(move, tictactoe)
* update_temp_history(tictactoe)
* train_model()

In [3]:
class model_creation:

    def __init__(self):
        self.moves = {(0,0):0,(0,1):0,(0,2):0,(1,0):0,(1,1):0,(1,2):0,(2,0):0,(2,1):0,(2,2):0}
        self.temp_history = pd.DataFrame(columns = [0,1,2,3,4,5,6,7,8,'move','outcome'])
        self.header = True
        self.history = None
        # Different types of models.
        self.model = None
        self.dt = None
        self.knn = None
        self.xgb = None
        self.nb = None

    def update(self, move, tictactoe, data_exists):
        # Update the statistics for winners.

        # Win = 1
        if move == 'x' and tictactoe.winner():
            self.temp_history['outcome'] = 1
        # Tie = 0
        elif tictactoe.draw():
            self.temp_history['outcome'] = 0
        # Loss = 0
        else:
            self.temp_history['outcome'] = -1

        # Write to csv and then reset temp_history
        if self.header and not data_exists:
            self.temp_history.to_csv('data.csv',index=False)
        else:
            with open('data.csv', 'a') as f:
                self.temp_history.to_csv(f, header = False, index = False)

        self.temp_history = self.temp_history.iloc[0:0]

    def update_temp_history(self, tictactoe):
        # Update temporary board state and moves.

        temp = {}
        # Update the state of the board.
        for i, row in enumerate(tictactoe.board):
            for j, col in enumerate(row):
                if tictactoe.board[i][j] == '-':
                    temp[i*3+j] = 0
                elif tictactoe.board[i][j] == 'x':
                    temp[i*3+j] = 1
                else:
                    temp[i*3+j] = 2
        # Set the move.
        temp['move'] = tictactoe.get_row()*3 + tictactoe.get_col()
        # We do not know the outcome yet. Randomly set to None.
        temp['outcome'] = None

        # Append the old temp_hisotry with the next temp_history.
        self.temp_history = self.temp_history.append(temp, ignore_index = True)

    def train_model(self, model_choice):
        # Train our model with the data found in data.csv.
        self.history = pd.read_csv('data.csv')

        data = self.history[['0','1','2','3','4','5','6','7','8', 'move']]
        target = self.history['outcome']
        target = target.astype('int')

        # Split the training data into to sets.
        x_train, x_test, y_train, y_test = train_test_split(data.values, target.values, test_size = 0.5)

        # Decide which model to train...

        # Decision Tree Classifier
        if model_choice == "dt":
            print("Decision Tree Classifier")
            self.dt = tree.DecisionTreeClassifier()
            self.dt = self.dt.fit(x_train, y_train)
            outTree = self.dt.predict(x_test)
            self.model = self.dt
            print("Accuracy for Decision Tree Classifier: " + str(accuracy_score(y_test, outTree)*100)+"%\n")

        # KNN
        elif model_choice == "knn":
            print("K-Nearest Neighbors")
            k_range = range(1,20)
            scores = {}
            scores_list = []
            for k in k_range:
                knn = KNeighborsClassifier(n_neighbors=k)
                knn.fit(x_train, y_train)
                y_pred = knn.predict(x_test)
                scores[k] = accuracy_score(y_test, y_pred)
                print(f"K = {k} accuracy = {accuracy_score(y_test, y_pred)}")
                scores_list.append(accuracy_score(y_test, y_pred))
            max_knn = max(scores.items(), key = operator.itemgetter(1))[0]
            print(f"Max Accuracy for KNN: K = {max_knn} Score = {scores[max_knn]}\n")
            self.knn = KNeighborsClassifier(n_neighbors=max_knn)
            self.knn.fit(x_train, y_train)
            self.model = self.knn

        ## XGBoost
        elif model_choice == "xgb":
            print("XGBoost")
            self.xgb = XGBClassifier()
            self.xgb = self.xgb.fit(x_train, y_train)
            y_pred = self.xgb.predict(x_test)
            predictions = [round(value) for value in y_pred]
            accuracy = accuracy_score(y_test, predictions)
            print("Accuracy for XGBoost: " + str(accuracy * 100) + "%\n")
            self.model = self.xgb

        ## Naive Bayes
        elif model_choice == "nbayes":
            print("Naive Bayes")
            self.nb = GaussianNB()
            self.nb = self.nb.fit(x_train, y_train)
            y_pred = self.nb.predict(data.values)
            print(f"Number of mislabled points for GaussianNB: {(target.values != y_pred).sum()} out of {data.shape[0]}")
            print(f"Accuracy is {(data.shape[0] - (target.values != y_pred).sum()) / data.shape[0] * 100}%\n")
            self.model = self.nb

        ## Model not implemented.
        else:
            raise ValueError(f"{model_choice} not yet implemented")


## Move methods
The move methods generate a move based on different methods.
The following methods are included:
* random_move(tictactoe)
* smart_move(tictactoe, statistics, move)
* heuristic_move(ttt, move)
* human_move(tictactoe)

In [4]:
def random_move(tictactoe):
    # Generate a random and valid move.
     row = random.randint(0,2)
     col = random.randint(0,2)
     tictactoe.set_row_col(row,col)
     while not tictactoe.valid_move():
         row = random.randint(0,2)
         col = random.randint(0,2)
         tictactoe.set_row_col(row,col)

def smart_move(tictactoe,statistics,move):
    # Generate a smart move that uses the trained model.
    # Convert dict_board to numpy array.
    dict_board = tictactoe.board_as_dict()
    lst_board = []
    for i in dict_board:
        lst_board += [dict_board[i]]
    arr_board = np.array(lst_board)

    # Get available moves.
    moves = tictactoe.get_available_moves()

    # Define list of numpy arrays for data.
    data_lst = []

    # Fill data_lst to test different combinations.
    for move in moves:
        data_lst += [np.append(arr_board, np.array([move]))]

    # Define a list that will hold our potential moves.
    viable_moves = []
    prediction_to_beat = -2

    # Make a prediction for each set of data and move.
    for data in data_lst:
        prediction = statistics.model.predict(np.array([data]))
        predict = sum(prediction)

        # Better prediction - overwrite the previous moves.
        if predict > prediction_to_beat:
            prediction_to_beat = predict
            viable_moves.clear()
            viable_moves += [(data[len(data)-1], predict)]

        # Same prediction - add to list of moves.
        elif predict == prediction_to_beat:
            # Store the move.
            viable_moves += [(data[len(data)-1], predict)]

    # Print out the model prediction.
    if prediction_to_beat == -1:
        print("Model believes this game will lose...")
    elif prediction_to_beat == 0:
        print("Model believes this game will tie...")
    else:
        print("Model believes this game will win...")

    # Randomly select from the viable moves
    random.shuffle(viable_moves)
    print("Randomly selecting from the following moves...", viable_moves)
    move = viable_moves[0][0]
    print("Move selected...", move)

    # Convert to row, col.
    col = move%3
    row = (move - col) / 3

    # Set the move
    tictactoe.set_row_col(int(row),(col))

def heuristic_move(ttt, player):
    # Heuristic -
    ## 1. Select a move if it results in a win.
    ## 2. Block an opponent from winning.
    tic = tictactoe()
    tic.board = copy.deepcopy(ttt.board)
    moves = tic.get_available_moves()

    # Select a move if it wins results in a win.
    for move in moves:
        tic.board = copy.deepcopy(ttt.board)
        col = int(move%3)
        row = int((move - col) / 3)
        tic.set_row_col(row,col)
        tic.set_move(player)
        if tic.winner():
            ttt.set_row_col(row,col)
            return

    # Set the opponent player.
    opponent = 'o'
    if player == 'o':
        opponent = 'x'

    # Check if opponent will win in any spot. If so, select that move.
    for move in moves:
        tic.board = copy.deepcopy(ttt.board)
        col = int(move%3)
        row = int((move - col) / 3)
        tic.set_row_col(row,col)
        tic.set_move(opponent)
        if tic.winner():
            ttt.set_row_col(row,col)
            return

    # Select a random move if no moves are beneficial.
    random_move(ttt)

def human_move(tictactoe):
    # Prompt the user for a move.
    available_moves = tictactoe.get_available_moves()
    formatted_moves = []
    for move in available_moves:
        col = int(move%3)
        row = int((move - col) / 3)
        formatted_moves += [f"[{row},{col}]"]

    # Ask for a move to be inputted and check bad cases.
    print(f"Your available moves are: {', '.join(formatted_moves)}")
    move = input("Please input your move in the form of row,col: ")

    # Make sure the input is not weird.
    good_input = False
    while not good_input:
        if len(move) != 3:
            move = input("Opps you messed up! Retype: ")
        else:
            good_input = True

    # Set the move and make sure the move is valid.
    row = int(move[0])
    col = int(move[2])
    tictactoe.set_row_col(row, col)
    while not tictactoe.valid_move():
        move = input("Invalid move, please input a new move: ")
        row = int(move[0])
        col = int (move[2])
        tictactoe.set_row_col(row,col)


## Game loop 
These methods run the actual game.
The following methods are included:
* game_loop(tictactoe, statistics, move, train, loops, data_exists)
* finish_game(tictactoe, statistics, move, train, data_exists)

In [5]:
def game_loop(tictactoe, statistics, train, loops = 10000, data_exists = True):
    # Game loop where tic-tac-toe is played.

    move = 'o' # x goes first
    exit = False
    i = 0
    print_counter = 0

    while i <= loops and exit == False:
        # Print counter to track how much data has been collected.
        if print_counter == 1000:
            print(i)
            print_counter = 0
        print_counter += 1

        # Change the move.
        if move == 'x':
            move = 'o'
        else:
            move = 'x'

        # Collect data first if training is True.
        if train:
            # Select a heuristic move (Might simulate a move that a human would make)
            heuristic_move(tictactoe, move)
            # Only record the stastics for 'x' moves.
            if move == 'x':
                statistics.update_temp_history(tictactoe)
            tictactoe.set_move(move)

        # Play a human against against a "smart" computer
        else:
            tictactoe.print_board()
            # If 'x' select a smart move.
            if move == 'x':
                print("It is x turn.\n")
                smart_move(tictactoe, statistics, move)

            # If 'o' Human selects a move.
            else:
                print("It is o turn.\n")
                human_move(tictactoe)
            tictactoe.set_move(move)

        # Determine if there is a winner or a tie.
        if tictactoe.winner() or tictactoe.draw():
            finish_game(tictactoe, statistics, move, train, data_exists)
            tictactoe.clear_board()
            move = 'o'
            if not train:
                quit = input("Do you want to keep playing? yes/no: ")
                if quit.lower()[0] == 'n':
                    exit = True
        i = i + 1

def finish_game(tictactoe, statistics, move, train, data_exists):
    # Finish the game and update statistics.
    if not train:
        tictactoe.print_board()
    statistics.update(move, tictactoe, data_exists)
    statistics.header = False


## Data Collection
The next cell contains code that collects data for randomly generated tictactoe moves.

>Note: There already exists a data set in data.csv. We will use this data set here, so we don't have to collect data.

In [6]:
DATA_EXISTS = True # Is there a csv file that exists with data in it?
TRAIN = False # Should we Train and Collect Data?
LOOP = 1000 # Number of moves to collect data for.

# Instance of tic-tac-toe and model.
ttt = tictactoe()
statistics = model_creation()

# Train if we want to collect Data.
if TRAIN:
    print(f"Collecting data... Running {LOOP} turns. Please wait...")
    game_loop(ttt, statistics, TRAIN, loops = LOOP, data_exists = DATA_EXISTS)

## What does the data look like?
Below displays the first five rows of our data set.

* Columns 0-8 represent the board. 
    * 0 represents an empty space.
    * 1 represents 'x'
    * 2 represents 'o'

* Move represents which spot 'x' went in.

* Outcome represents whether the board state and move selected ended in a win, tie, or loss.
    * 1 indicates that 'x' won
    * 0 indicates a tie
    * -1 indicates that 'o' won

In [7]:
# Show what the data set looks like.
data = pd.read_csv('data.csv')
print("Head of data set.")
data.head()

Head of data set.


,0,1,2,3,4,5,6,7,8,move,outcome
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0
1,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,4.0,0
2,0.0,0.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0
3,0.0,1.0,2.0,2.0,1.0,0.0,1.0,2.0,0.0,8.0,0
4,2.0,1.0,2.0,2.0,1.0,0.0,1.0,2.0,1.0,5.0,0


## Modeling Training
The next cell trains several models based on the data that was collected.

In [8]:
# Train Model
print("Training model...\n")
"""
Select from the following models...
Decision Tree Classifier: model_choice = "dt"
KNN: model_choice = "knn"
XGBoost : model_choice = "xgb"
GuassianNB : model_choice = "nbayes"
"""
model_choice = "dt"
statistics.train_model(model_choice)

model_choice = "knn"
statistics.train_model(model_choice)

model_choice = "xgb"
statistics.train_model(model_choice)

model_choice = "nbayes"
statistics.train_model(model_choice)    

Training model...

Decision Tree Classifier
Accuracy for Decision Tree Classifier: 81.53843355854832%

K-Nearest Neighbors
K = 1 accuracy = 0.773690767398085
K = 2 accuracy = 0.7774554647218762
K = 3 accuracy = 0.7848939246514927
K = 4 accuracy = 0.7891223890369105
K = 5 accuracy = 0.7917413089143304
K = 6 accuracy = 0.7833389409742746
K = 7 accuracy = 0.7956060344278842
K = 8 accuracy = 0.7838390819230874
K = 9 accuracy = 0.793896461730124
K = 10 accuracy = 0.7875219380007092
K = 11 accuracy = 0.7922778237503296
K = 12 accuracy = 0.7910229246423992
K = 13 accuracy = 0.7901135774627395
K = 14 accuracy = 0.7943511353199538
K = 15 accuracy = 0.7976975329411016
K = 16 accuracy = 0.7927870581709391
K = 17 accuracy = 0.7922960106939229
K = 18 accuracy = 0.7913502896270768
K = 19 accuracy = 0.7921505151451773
Max Accuracy for KNN: K = 15 Score = 0.7976975329411016

XGBoost
Accuracy for XGBoost: 67.69180405386973%

Naive Bayes
Number of mislabled points for GaussianNB: 111241 out of 219938
Ac

## tic-tac-toe
The following cells allows a human to play against each model that was trained. 

Two games are played against each model to judge their decision making.

In [9]:
# Human testing...
train = False

In [10]:
print("Decision Tree Model...")
statistics.model = statistics.dt
game_loop(ttt, statistics, train)

Decision Tree Model...
- - -
- - -
- - -
It is x turn.

Model believes this game will tie...
Randomly selecting from the following moves... [(7, 0), (8, 0), (1, 0), (6, 0), (0, 0), (3, 0), (5, 0), (2, 0), (4, 0)]
Move selected... 7
- - -
- - -
- x -
It is o turn.

Your available moves are: [0,0], [0,1], [0,2], [1,0], [1,1], [1,2], [2,0], [2,2]
Please input your move in the form of row,col: 1,1
- - -
- o -
- x -
It is x turn.

Model believes this game will tie...
Randomly selecting from the following moves... [(5, 0), (2, 0), (0, 0), (6, 0), (3, 0), (8, 0)]
Move selected... 5
- - -
- o x
- x -
It is o turn.

Your available moves are: [0,0], [0,1], [0,2], [1,0], [2,0], [2,2]
Please input your move in the form of row,col: 2,2
- - -
- o x
- x o
It is x turn.

Model believes this game will tie...
Randomly selecting from the following moves... [(0, 0), (2, 0), (3, 0), (1, 0), (6, 0)]
Move selected... 0
x - -
- o x
- x o
It is o turn.

Your available moves are: [0,1], [0,2], [1,0], [2,0]
Plea

In [11]:
print("K-Nearest Neighbors Model.../n")
statistics.model = statistics.knn
game_loop(ttt, statistics, train)

K-Nearest Neighbors Model.../n
- - -
- - -
- - -
It is x turn.

Model believes this game will tie...
Randomly selecting from the following moves... [(4, 0), (8, 0), (2, 0), (6, 0), (5, 0), (0, 0), (7, 0), (3, 0)]
Move selected... 4
- - -
- x -
- - -
It is o turn.

Your available moves are: [0,0], [0,1], [0,2], [1,0], [1,2], [2,0], [2,1], [2,2]
Please input your move in the form of row,col: 0,0
o - -
- x -
- - -
It is x turn.

Model believes this game will tie...
Randomly selecting from the following moves... [(5, 0), (2, 0), (6, 0), (1, 0), (7, 0), (8, 0), (3, 0)]
Move selected... 5
o - -
- x x
- - -
It is o turn.

Your available moves are: [0,1], [0,2], [1,0], [2,0], [2,1], [2,2]
Please input your move in the form of row,col: 1,0
o - -
o x x
- - -
It is x turn.

Model believes this game will win...
Randomly selecting from the following moves... [(2, 1)]
Move selected... 2
o - x
o x x
- - -
It is o turn.

Your available moves are: [0,1], [2,0], [2,1], [2,2]
Please input your move in th

In [12]:
print("XGBoost Model...")
statistics.model = statistics.xgb
game_loop(ttt, statistics, train)

XGBoost Model...
- - -
- - -
- - -
It is x turn.

Model believes this game will tie...
Randomly selecting from the following moves... [(1, 0), (2, 0), (0, 0), (5, 0), (6, 0), (7, 0), (4, 0), (3, 0), (8, 0)]
Move selected... 1
- x -
- - -
- - -
It is o turn.

Your available moves are: [0,0], [0,2], [1,0], [1,1], [1,2], [2,0], [2,1], [2,2]
Please input your move in the form of row,col: 1,1
- x -
- o -
- - -
It is x turn.

Model believes this game will tie...
Randomly selecting from the following moves... [(5, 0), (0, 0), (3, 0), (7, 0), (8, 0), (2, 0), (6, 0)]
Move selected... 5
- x -
- o x
- - -
It is o turn.

Your available moves are: [0,0], [0,2], [1,0], [2,0], [2,1], [2,2]
Please input your move in the form of row,col: 0,2
- x o
- o x
- - -
It is x turn.

Model believes this game will tie...
Randomly selecting from the following moves... [(7, 0), (3, 0), (8, 0), (6, 0)]
Move selected... 7
- x o
- o x
- x -
It is o turn.

Your available moves are: [0,0], [1,0], [2,0], [2,2]
Please inp

In [13]:
print("Naive Bayes Model...")
statistics.model = statistics.nb
game_loop(ttt, statistics, train)

Naive Bayes Model...
- - -
- - -
- - -
It is x turn.

Model believes this game will win...
Randomly selecting from the following moves... [(8, 1), (4, 1), (3, 1), (7, 1), (6, 1), (5, 1)]
Move selected... 8
- - -
- - -
- - x
It is o turn.

Your available moves are: [0,0], [0,1], [0,2], [1,0], [1,1], [1,2], [2,0], [2,1]
Please input your move in the form of row,col: 0,0
o - -
- - -
- - x
It is x turn.

Model believes this game will win...
Randomly selecting from the following moves... [(7, 1), (2, 1), (3, 1), (5, 1), (6, 1), (4, 1), (1, 1)]
Move selected... 7
o - -
- - -
- x x
It is o turn.

Your available moves are: [0,1], [0,2], [1,0], [1,1], [1,2], [2,0]
Please input your move in the form of row,col: 2,0
o - -
- - -
o x x
It is x turn.

Model believes this game will tie...
Randomly selecting from the following moves... [(2, 0), (5, 0), (4, 0), (3, 0), (1, 0)]
Move selected... 2
o - x
- - -
o x x
It is o turn.

Your available moves are: [0,1], [1,0], [1,1], [1,2]
Please input your move

## How did these models perform?
Of the four models I trained, the Decision Tree Classifier performed the best. Below is a slightly more in depth analysis of each model and its performance.
> Note: These models were trained on a data set with a size of ~220,000 data points.

### Decision Tree Classifier
Accuracy: ~81.5%
Human Game Results:
* Game One: Tie
* Game Two: X wins
The Descision Tree Classifier was able to identify moves that would likely result in a win. However, this model still missed some obvious moves in some situations.

### K-Nearest Neighbors
Accuracy: ~79.8% (K = 15)
Human Game Results:
* Game One: X loses
* Game Two: X loses
The K-Nearest Neighbors did not succesfully identify moves that would improve its chance to win. In fact, it missed moves that would allow the game to atleast tie.

### XGBoost
Accuracy: ~67.7%
Human Game Results:
* Game One: X loses
* Game Two: X loses
The XGBoost model did not perform well. Not only was the accuracy poor, but the model also missed obvious moves that should be made by a normal person.

### Naive Bayes
Accuracy: ~49.4%
Human Game Results:
* Game One: X loses
* Game Two: X loses
The Naive Bayes model performed, at best, randomly. The poor accuracy indicates that the model performed just as well as a random move generator. Both games indicated that.